In [11]:
import subprocess
import time
import shutil
import xml.etree.ElementTree as ET
from evaluation.metrics import calc_metrics_for_all_images
from statistics import mean, median
import os
import signal
import skopt
import wandb


In [12]:
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="old-vision-parameter-optimisation",

    # track hyperparameters and run metadata
    config={
        "metric": "f1",
        "f1_beta": 1,
        "epochs": 250,
        "minimizer": "gbrt"
    }
)

F1,▁▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▆▆▁▁▇▄▁█▇▁▇▁▁▁▁██
IoU,▁▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▅▆▁▁▇▃▁█▆▁▆▁▁▁▁██
Presicion,▆▁▆▆▆▆▆▆▆▆▆▆▆▆▆▇▆▆▇▆▄▆▆▃█▆▄▇▆▃▆▆▆▆▂▅
Recall,▁▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▆▆▁▁▆▄▁█▇▁▇▁▁▁▁██
V_COLORBORDER,▄▄▆▂▅█▄▇▂▇█▁█▃▄▃▆▆▅▂▂▃▇▁▅▅▅▃▂▆▂▄▄▂▂▂
V_GREENGAIN,▇▇▂█▅▆█▇▄▆▇▃▄▇▄▇▆▁▃▃██▃▆▆█▄▇▁▇▅▇▂▇█▂
V_GREENREGIONCOLORDIST,▄▄▇█▃▃▅▃▇▇▁▇▃▆▇▄▁▁█▇▅▃▃█▅▅▃█▇▁▆▁▆▁▃▆
V_LINESPACINGDOWN,▇▃▅▂▆▄█▇▇▃█▄▇▁▅▃▁▆▂▁▁▁▆▁▃▄▃▇▇▅▅█▅▇▃▃
V_LINESPACINGUP,▇▇▆▂▃▄▆▆▁▆█▄▅█▂▇▇▃▂▃▅▂▁▁▅▅▅▃█▃▇█▄▃▃▃
V_MAXEDGESINLINE,▆▆▅▃▅▆▇▇▆▄█▂▆▃▇▇▇▄▁▁▃▃▁▃▆▆▄▅▄▅▆▆▂▄▅▆
V_MAXLINEBORDER,▃█▁▄▁▆▁▅▃▄██▄██▇▄▃███▇██▆▅██████████


In [13]:
def execute_vison():
    # Define the command you want to execute
    command = "LD_LIBRARY_PATH=/home/ohm/Documents/HTWKRobots/hackingweekend/naoTeamRepo/firmware_5.0/build/build/vision/:/home/ohm/Documents/HTWKRobots/hackingweekend/naoTeamRepo/firmware_5.0/build/build/imagetools/:/home/ohm/Documents/HTWKRobots/hackingweekend/naoTeamRepo/firmware_5.0/build/build/option_framework/:/home/ohm/Documents/HTWKRobots/hackingweekend/naoTeamRepo/firmware_5.0/build/build/vision_primitives/:/home/ohm/Documents/HTWKRobots/hackingweekend/naoTeamRepo/firmware_5.0/build/build/visualizer/:/home/ohm/Documents/HTWKRobots/hackingweekend/naoTeamRepo/firmware_5.0/build/build/flightrecorder/client/:/home/ohm/Documents/HTWKRobots/hackingweekend/naoTeamRepo/firmware_5.0/build/build/naomessages/:/home/ohm/Documents/HTWKRobots/hackingweekend/naoTeamRepo/firmware_5.0/build/build/sockethelper/:/home/ohm/Documents/HTWKRobots/hackingweekend/naoTeamRepo/firmware_5.0/build/build/vision/tfliteexecuter/:/home/ohm/Documents/HTWKRobots/hackingweekend/naoTeamRepo/firmware_5.0/build/build/cam_utils/:/home/ohm/Documents/HTWKRobots/hackingweekend/naoTeamRepo/firmware_5.0/build/build/cam_utils/libCameraUtils.so:/home/ohm/Documents/HTWKRobots/hackingweekend/naoTeamRepo/firmware_5.0/build/build/utils/:/home/ohm/Documents/HTWKRobots/hackingweekend/naoTeamRepo/firmware_5.0/cmake-build-debug/3rdparty/easy_profiler/:/home/ohm/Documents/HTWKRobots/hackingweekend/naoTeamRepo/firmware_5.0/build/build/3rdparty/liblodepng/ /home/ohm/Documents/HTWKRobots/hackingweekend/naoTeamRepo/firmware_5.0/build/build/vision/demo/demo -d /home/ohm/PycharmProjects/FiFi/dataset/images"

    # Start the process
    process = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True, preexec_fn=os.setsid)

    # Wait for 5 seconds
    time.sleep(5)

    os.killpg(os.getpgid(process.pid), signal.SIGTERM)


In [14]:
def evaluate(params):
    # copy and replace anotations file from dataset to /home/ohm/Documents/HTWKRobots/hackingweekend/naoTeamRepo/firmware_5.0/vision/demo/
    shutil.copyfile("/home/ohm/PycharmProjects/FiFi/dataset/annotations.xml", "/home/ohm/Documents/HTWKRobots/hackingweekend/naoTeamRepo/firmware_5.0/vision/demo/annotations.xml")

    # convert all ints in params to strings
    params = [str(int) for int in params]

    # env viriables for region classifier
    os.environ["V_TEDGEUP"] = params[0]#"12"
    os.environ["V_TEDGEDOWN"] = params[1]#"16"
    os.environ["V_LINESPACINGUP"] = params[2]#"10"
    os.environ["V_LINESPACINGDOWN"] = params[3]#"20"
    os.environ["V_MAXEDGESINLINE"] = params[4]#"3"
    os.environ["V_GREENREGIONCOLORDIST"] = params[5]#"360"
    os.environ["V_MAXLINEBORDER"] = params[6]#"6"

    # env variables for field color detector
    os.environ["V_MINFIELDAREA"] = params[7]#"500"
    os.environ["V_PIXELSPACING"] = params[8]#"16"
    os.environ["V_COLORBORDER"] = params[9]#"8"
    os.environ["V_GREENGAIN"] = params[10]#"2"

    execute_vison()

    # Parse the XML file
    xml_file = '/home/ohm/Documents/HTWKRobots/hackingweekend/naoTeamRepo/firmware_5.0/vision/demo/annotations.xml'
    tree = ET.parse(xml_file)
    root = tree.getroot()

    presicion, recall, iou, f1 = calc_metrics_for_all_images(root, "/home/ohm/PycharmProjects/FiFi/dataset/images/")

    print("F1: " + str(f1))
    print("Presicion: " + str(presicion))
    print("Recall: " + str(recall))
    print("IoU: " + str(mean(iou)))


    return f1, presicion, recall, mean(iou)
    #print("Presicion: " + str(presicion))
    #print("IoU: " + str(mean(iou)))


In [15]:
SPACE = [
    skopt.space.Integer(1, 30, name='V_TEDGEUP'),
    skopt.space.Integer(1, 30, name='V_TEDGEDOWN'),
    skopt.space.Integer(1, 30, name='V_LINESPACINGUP'),
    skopt.space.Integer(1, 30, name='V_LINESPACINGDOWN'),
    skopt.space.Integer(1, 10, name='V_MAXEDGESINLINE'),
    skopt.space.Integer(50, 800, name='V_GREENREGIONCOLORDIST'),
    skopt.space.Integer(1, 20, name='V_MAXLINEBORDER'),
    skopt.space.Integer(100, 1000, name='V_MINFIELDAREA'),
    skopt.space.Integer(1, 30, name='V_PIXELSPACING'),
    skopt.space.Integer(1, 20, name='V_COLORBORDER'),
    skopt.space.Real(1, 5, name='V_GREENGAIN', prior='uniform')
]

opt = skopt.Optimizer(SPACE, n_initial_points=10, acq_func="EI", acq_optimizer="auto", random_state=0)

best_score = 0
best_params = None
for i in range(500):
    params = opt.ask()
    f1, presicion, recall, iou = evaluate(params)

    if f1 > best_score:
        best_score = f1
        best_params = params

    wandb.log({
        "best_score": best_score,
        "F1": f1,
        "Presicion": presicion,
        "Recall": recall,
        "IoU": iou,
        "V_TEDGEUP": params[0],
        "V_TEDGEDOWN": params[1],
        "V_LINESPACINGUP": params[2],
        "V_LINESPACINGDOWN": params[3],
        "V_MAXEDGESINLINE": params[4],
        "V_GREENREGIONCOLORDIST": params[5],
        "V_MAXLINEBORDER": params[6],
        "V_MINFIELDAREA": params[7],
        "V_PIXELSPACING": params[8],
        "V_COLORBORDER": params[9],
        "V_GREENGAIN": params[10]
    })

    opt.tell(params, -1*f1)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
Error connect socket: : Connection refused
Error: bind to port 40588 on 127.0.0.1 failed in file ../../sockethelper/sockethelper.cpp line 51. Exiting.
Segmentation fault
/home/ohm/PycharmProjects/FiFi/venv/lib/python3.8/site-packages/shapely/set_operations.py:426: RuntimeWarning: divide by zero encountered in unary_union
  return lib.unary_union(collections, **kwargs)


F1: 0.08904158516053173
Presicion: 0.9528132492669349
Recall: 0.04670302029490031
IoU: 0.05793207074615064


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
Error connect socket: : Connection refused
Error: bind to port 40588 on 127.0.0.1 failed in file ../../sockethelper/sockethelper.cpp line 51. Exiting.
Error connect socket: : Connection refused
Error: bind to port 40588 on 127.0.0.1 failed in file ../../sockethelper/sockethelper.cpp line 51. Exiting.
Error connect socket: : Connection refused
Error: bind to port 40588 on 127.0.0.1 failed in file ../../sockethelper/sockethelper.cpp line 51. Exiting.
Error connect socket: : Connection refused
Error: bind to port 40588 on 127.0.0.1 failed in file ../../sockethelper/sockethelper.cpp line 51. Exiting.


F1: 0.25713898219271153
Presicion: 0.9070208269808568
Recall: 0.14980410561724145
IoU: 0.200301258729169


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


...

F1: 0.08904158516053173
Presicion: 0.9528132492669349
Recall: 0.04670302029490031
IoU: 0.05793207074615064


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
Error connect socket: : Connection refused
Error: bind to port 40588 on 127.0.0.1 failed in file ../../sockethelper/sockethelper.cpp line 51. Exiting.
Error connect socket: : Connection refused
Error: bind to port 40588 on 127.0.0.1 failed in file ../../sockethelper/sockethelper.cpp line 51. Exiting.
Error connect socket: : Connection refused
Error: bind to port 40588 on 127.0.0.1 failed in file ../../sockethelper/sockethelper.cpp line 51. Exiting.
Error connect socket: : Connection refused
Error: bind to port 40588 on 127.0.0.1 failed in file ../../sockethelper/sockethelper.cpp line 51. Exiting.
/home/ohm/PycharmProjects/FiFi/venv/lib/python3.8/site-packages/shapely/set_operations.py:426: RuntimeWarning: divide by zero encountered in unary_union
  return lib.unary_union(collections, **kwargs)


F1: 0.3060468250034743
Presicion: 0.9332575271075725
Recall: 0.18303512864607208
IoU: 0.2354162403908405


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
Error connect socket: : Connection refused
Error: bind to port 40588 on 127.0.0.1 failed in file ../../sockethelper/sockethelper.cpp line 51. Exiting.
Segmentation fault
/home/ohm/PycharmProjects/FiFi/venv/lib/python3.8/site-packages/shapely/set_operations.py:426: RuntimeWarning: divide by zero encountered in unary_union
  return lib.unary_union(collections, **kwargs)


F1: 0.08904158516053173
Presicion: 0.9528132492669349
Recall: 0.04670302029490031
IoU: 0.05793207074615064


In [18]:
print(best_score)

0.25315602994645053


In [19]:
print(evaluate(best_params))

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
Error connect socket: : Connection refused
Error: bind to port 40588 on 127.0.0.1 failed in file ../../sockethelper/sockethelper.cpp line 51. Exiting.
Error connect socket: : Connection refused
Error: bind to port 40588 on 127.0.0.1 failed in file ../../sockethelper/sockethelper.cpp line 51. Exiting.
Error connect socket: : Connection refused
Error: bind to port 40588 on 127.0.0.1 failed in file ../../sockethelper/sockethelper.cpp line 51. Exiting.
Error connect socket: : Connection refused
Error: bind to port 40588 on 127.0.0.1 failed in file ../../sockethelper/sockethelper.cpp line 51. Exiting.
/home/ohm/PycharmProjects/FiFi/venv/lib/python3.8/site-packages/shapely/set_operations.py:426: RuntimeWarning: divide by zero encountered in unary_union
  return lib.unary_union(collections, **kwargs)
/home/ohm/PycharmProjects/FiFi/venv/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in

F1: 0.31634278361549195
Presicion: 0.9246858537487302
Recall: 0.19081029221811519
IoU: 0.25258395565384417
(0.31634278361549195, 0.9246858537487302, 0.19081029221811519, 0.25258395565384417)


In [16]:
wandb.run.summary["best_params"] = best_params
wandb.run.summary["best_score"] = best_score

In [17]:
wandb.finish()

F1,▁▁▁▁▄▅▇▄▄▅▆▁▄▄▅▁▃▁▇▇▇▇▆▂▇▇█▅▁█▆█▁▁▁▁▁▄▆█
IoU,▁▁▁▁▄▄▆▄▅▅▆▁▃▄▅▁▄▁▇▇▇▇▅▂▆▇▇▄▁█▇█▁▁▁▁▁▄▇█
Presicion,▁▁▁▁▅▅▇▅▅▆▆▁▄▄▆▁▄▁▇▇▇▇▅▂▆▇▇▄▁█▇█▁▁▁▁▁▄▇█
Recall,████▇▂▇█▇█▇█▄▆████▇▅▆▆▅▇▇▇█▆█▇▇█▁████▇▆▇
V_COLORBORDER,▂███▅▅▁▅▆▄▃▂█▇▅▇▆▄▁▆▇▄█▇▄▅▆▄▇█▄▅█▄▃▁▃▂█▄
V_GREENGAIN,█▄▁▃▅▅▄▄▃▆▁▆▂▇▂▇▁▃▃▅▄▄▆▄▅▇▃▂▂▂▆▃▅▄▆▇▅▆▁▁
V_GREENREGIONCOLORDIST,▇▅▁▃▆▆▄▁▃▅▂▅█▁▆▄▄▁▃▂▇▃▅▃██▂▃▆▂▄█▇▄▄▄▂▄▅▅
V_LINESPACINGDOWN,▂▃▃▂▁▁▄▁▁▃▁▁▁▁▂█▁▃▁▂▁▂▃▄▂▃▃▃▃▃▁▃█▇▅▇▇▁▃▃
V_LINESPACINGUP,▂▅▄▅▁▁▁▁▁▁▁▁▁▃▁█▁▁▃▃▃▃▁▃▂▁▂▃▅▃▃▃▂█▃▃▂▁▃▄
V_MAXEDGESINLINE,▃█▅▅▆▁▆▂▃▇▁▅▅▄▅▃▃▅▃▅▇▃▄▁▂▅▅▄▆▆▄▅▃▇▁▂▄▅█▇
V_MAXLINEBORDER,▄██▁▆█▃▁▃▁▅▆▆▂▂▄▅▆▇██▇▂▁▁▆▇▇▅▇▆█▁█▅██▂▇▇
